In [15]:
!pip install refractio[snowflake]
!pip install scikit-learn 
!pip install matplotlib
!pip install statsmodels
!pip install pmdarima
!pip install fosforml

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 40kB 5.1MB/s eta 0:00:011
     |████████████████████████████████| 10.9MB 43.6MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 78.3MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 63.1MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 94kB/s /s eta 0:00:01███                | 99.8MB 85.0MB/s eta 0:00:02�████████▌           | 128.0MB 112.4MB/s eta 0:00:01█████▏        | 145.1MB 112.4MB/s eta 0:00:01
     |████████████████████████████████| 98.2MB 231kB/s  eta 0:00:01��██████████   | 89.3MB 80.1MB/s eta 0:00:01
     |████████████████████████

In [19]:
pip install --upgrade pip

     |████████████████████████████████| 1.8MB 8.1MB/s eta 0:00:01
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Note: you may need to restart the kernel to use updated packages.


In [16]:
## Step 1. Connecting to the Snowflake Connection specified in Insight Designer Module

In [8]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [9]:
### (i) Getting dataset from Insight Designer

In [10]:
table_name = 'PLANT_GROWTH_DATA'

Exception occurred in reading data_frame from snowflake connection: HTTPConnectionPool(host='fdc-project-manager', port=80): Max retries exceeded with url: /project-manager/connections/api/External/v2/external/getConnConfig/PHARMACEUTICAL_BRAND/refract_user/3ed9c58f-21bc-45b0-bd1f-62e2bd52c1d6 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f62132c8ac0>: Failed to establish a new connection: [Errno -2] Name or service not known'))
